In [ ]:
pip install mediapipe

In [ ]:
#First run this cell in order to get frames of video.
# it saves frames into folder names like video name+opencv
#after running first cell, please run second cell to see result


from datetime import timedelta
import cv2
import numpy as np
import os

# i.e if video of duration 30 seconds, saves 10 frame per second = 300 frames saved in total
SAVING_FRAMES_PER_SECOND = 10

def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05)
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")


def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s


def main(video_file,filename):
    # make a folder by the name of the video file
    if not os.path.isdir(filename):
        os.mkdir(filename)
    # read the video file
    cap = cv2.VideoCapture(video_file)
    # get the FPS of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # get the list of duration spots to save
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    # start the loop
    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break
        # get the duration by dividing the frame count by the FPS
        frame_duration = count / fps
        try:
            # get the earliest duration to save
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # the list is empty, all duration frames were saved
            break
        if frame_duration >= closest_duration:
            # if closest duration is less than or equals the frame duration,
            # then save the frame
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            cv2.imwrite(os.path.join(filename, f"frame{frame_duration_formatted}.jpg"), frame)
            # drop the duration spot from the list, since this duration spot is already saved
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # increment the frame count
        count += 1
        



if __name__ == "__main__":
    import sys
    #write your file path here
    video_file = "/content/2022-10-21 19-22-27.mkv"
    filename, _ = os.path.splitext(video_file)
    filename += "-opencv"
    main(video_file,filename)


In [ ]:
#run this cell to see if picture contains hand or not. It can contain both hands
from google.colab.patches import cv2_imshow
import mediapipe as mp
from os import listdir
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
mp_model = mp_hands.Hands(
    static_image_mode=True, # only static images
    max_num_hands=2, # max 2 hands detection
    min_detection_confidence=0.5) # detection confidence
    


list1=[]
folder_dir = filename
for image in os.listdir(folder_dir):
  if (image.endswith(".png") or image.endswith(".jpg")\
        or image.endswith(".jpeg")):
    path=image
    if os.path.exists(folder_dir+"/"+image):
      image = cv2.imread(folder_dir+"/"+image)
    # now we flip image and convert to rgb image and input to model
      image = cv2.flip(image, 1)
    
      results = mp_model.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      print(path,results.multi_handedness,sep="      ")
    else:
        print("Empty image")
    

#none means no hand
# "label": Right -> right hand
#"label": Left-> left hand